# Price Predictor Supervised Model

In [3]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from datetime import timedelta
from sqlalchemy import create_engine
import psycopg2
from config import db_password

## Load in data

In [4]:
# database address
host_name = 'database-1.cvhixt1fojqo.us-east-1.rds.amazonaws.com'

In [5]:
# create the database engine
db_string = f"postgresql://postgres:{db_password}@{host_name}:5432/Crypto_Data"

In [6]:
engine = create_engine(db_string)

In [7]:
crypto_df = pd.read_sql("SELECT * FROM crypto_orig", con=engine)

In [8]:
crypto_df.shape

(5012, 9)

In [9]:
crypto_df.dtypes

type_crypto       object
date_orig         object
daystarttoend      int64
open_            float64
high_            float64
low_             float64
close_           float64
volume_          float64
marketcap_       float64
dtype: object

In [11]:
crypto_df.sample(10)

,type_crypto,date_orig,daystarttoend,open_,high_,low_,close_,volume_,marketcap_
3567,xrp,2018-05-13,224,0.684981,0.739311,0.672096,0.729610,4.191308e+08,2.809678e+10
1701,eth,2017-11-04,34,305.968210,305.968210,296.862375,300.501831,4.345725e+08,2.875765e+10
2488,eth,2019-12-31,821,131.703316,133.064496,129.030086,129.835250,7.739192e+09,1.426339e+10
4997,xrp,2022-04-12,1654,0.695203,0.720916,0.694100,0.717054,2.908405e+09,3.255490e+10
2824,eth,2020-12-01,1157,616.183534,634.754677,573.516677,589.814467,2.780131e+10,6.856145e+10
4725,xrp,2021-07-14,1382,0.619734,0.624013,0.590476,0.618632,2.799448e+09,2.816377e+10
1505,ada,2021-11-15,1506,2.041545,2.090438,2.018552,2.021545,1.601441e+09,6.446874e+10
24,ada,2017-10-26,25,0.027344,0.027485,0.026423,0.027193,2.307320e+06,6.978341e+08
571,ada,2019-04-26,572,0.069651,0.070611,0.066818,0.069183,1.366000e+08,1.778018e+09
3236,eth,2022-01-17,1569,3351.399314,3356.418118,3156.926754,3214.910372,1.417366e+10,3.890000e+11


## Transform Datatypes

In [12]:
crypto_df['date_orig'] = pd.to_datetime(crypto_df["date_orig"])
crypto_df.dtypes

type_crypto              object
date_orig        datetime64[ns]
daystarttoend             int64
open_                   float64
high_                   float64
low_                    float64
close_                  float64
volume_                 float64
marketcap_              float64
dtype: object

## Split Data

In [14]:
crypto_list = crypto_df['type_crypto'].unique()
crypto_list

array(['ada', 'eth', 'xrp'], dtype=object)

## ML Model Functions

In [31]:
def pricepredict(df, dt):
    
    # number of future days to predict
    dt_days = dt
    
    # initialize dt column for how far we want to predict
    df['date_pred'] = df['date_orig'] + timedelta(days=dt_days)
    
    # create a copy dataframe to pull target close price
    df_copy = df.copy()
    
    # merge dataframes to get target variable
    df_clean = df.merge(df_copy, how='left', left_on='date_pred', right_on='date_orig', suffixes=(None, '_pred'))

    # drop empty rows
    df_pre_final = df_clean.dropna()
    
    # FOR POSTGRES "crypto_clean" table
    clean_df = df_pre_final[['type_crypto','date_orig','daystarttoend','open_','high_','low_','close_','volume_','marketcap_','date_pred','close__pred']]
    
    # only include relevant features (X) and target variables (y)
    df_final = df_pre_final[['open_','high_','low_','close_','volume_','marketcap_','close__pred']]
    
    # features
    X = df_final.drop('close__pred', axis=1)
    X = X.values.reshape(-1,6)
    
    # target
    y = df_final['close__pred']
    
    # initialize class for linear reg algorithms
    model = LinearRegression()
    
    # fit or train the data
    model.fit(X,y)
    
    # predict outcomes before current date
    y_pred = model.predict(X)
    
    # test data
    df_test = df_clean.loc[pd.isna(df_clean["type_crypto_pred"]),:]
    
    # test features
    X_test = df_test[['open_','high_','low_','close_','volume_','marketcap_']]
    X_test = X_test.values.reshape(-1,6)
    
    # test predictions
    y_pred_future = model.predict(X_test)
    
    # creating output dataframe
    close_pred_series = pd.Series(y_pred).append(pd.Series(y_pred_future))
    close_pred_df = pd.DataFrame(close_pred_series, columns=['close_'])
    close_pred_df.reset_index(inplace=True)
    predict_df = pd.DataFrame(df_clean['type_crypto'])
    predict_df['date_orig'] = df_clean['date_orig']
    predict_df['close_orig'] = df_clean['close_']
    predict_df['date_pred'] = df_clean['date_pred']
    predict_df['close_pred'] = close_pred_df['close_']
    
    return clean_df, predict_df

In [38]:
#initial output tables
clean_df_60_day = pd.DataFrame()
predict_df_60_day = pd.DataFrame()
clean_df_30_day = pd.DataFrame()
predict_df_30_day = pd.DataFrame()

In [41]:
# for loop to separate models
for crypto in crypto_list:
    
    # run the ML models
    df = crypto_df.loc[crypto_df['type_crypto'] == crypto,:]
    clean_60_day, predict_60_day = pricepredict(df, 60)
    clean_30_day, predict_30_day = pricepredict(df, 30)
    
    # create the output dataframes
    clean_df_60_day = clean_df_60_day.append(clean_60_day, ignore_index=True)
    predict_df_60_day = predict_df_60_day.append(predict_60_day, ignore_index=True)
    clean_df_30_day = clean_df_30_day.append(clean_30_day, ignore_index=True)
    predict_df_30_day = predict_df_30_day.append(predict_30_day, ignore_index=True)

/Users/briancha/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/briancha/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/briancha/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [44]:
clean_df_60_day.sample(15)

,type_crypto,date_orig,daystarttoend,open_,high_,low_,close_,volume_,marketcap_,date_pred,close__pred
3749,xrp,2019-03-11,526,0.311394,0.313946,0.308788,0.310669,5.428024e+08,1.286338e+10,2019-05-10,0.300907
4728,xrp,2021-11-14,1505,1.190883,1.198676,1.170884,1.188374,2.405261e+09,5.473511e+10,2022-01-13,0.770131
1706,eth,2018-01-08,99,1118.410763,1352.348289,1044.903400,1200.982949,6.782506e+09,1.210000e+11,2018-03-09,730.711354
4106,xrp,2020-03-02,883,0.227639,0.241071,0.226507,0.238988,1.327213e+09,1.014844e+10,2020-05-01,0.217508
760,ada,2019-11-01,761,0.041249,0.042292,0.040908,0.042117,9.766154e+07,1.080855e+09,2019-12-31,0.032899
4044,xrp,2019-12-31,821,0.193063,0.193614,0.190184,0.192921,7.451743e+08,8.317454e+09,2020-02-29,0.230269
1776,eth,2018-03-19,169,537.513155,557.659260,518.694116,555.268133,2.567576e+09,5.399218e+10,2018-05-18,693.773754
4155,xrp,2020-04-20,932,0.189462,0.194130,0.179867,0.182250,1.314586e+09,8.177210e+09,2020-06-19,0.186939
1070,ada,2020-09-06,1071,0.091333,0.095204,0.087472,0.093568,6.302454e+08,2.906527e+09,2020-11-05,0.097598
4770,xrp,2021-12-26,1547,0.926157,0.933052,0.908870,0.920371,1.575472e+09,4.237951e+10,2022-02-24,0.694914


In [45]:
predict_df_60_day.sample(15)

,type_crypto,date_orig,close_orig,date_pred,close_pred
1764,eth,2018-01-06,1004.694291,2018-03-07,941.140188
1152,ada,2020-11-27,0.140312,2021-01-26,0.338659
3447,xrp,2018-01-13,2.478072,2018-03-14,0.746850
169,ada,2018-03-20,0.199989,2018-05-19,0.187343
601,ada,2019-05-26,0.086805,2019-07-25,0.171753
2876,eth,2021-01-22,1237.203403,2021-03-23,1766.806165
4251,xrp,2020-03-27,0.174437,2020-05-26,0.397174
3243,eth,2022-01-24,2448.138000,2022-03-25,2267.095099
2339,eth,2019-08-04,221.665372,2019-10-03,359.350740
4666,xrp,2021-05-16,1.450000,2021-07-15,0.782777


In [46]:
clean_df_30_day.sample(15)

,type_crypto,date_orig,daystarttoend,open_,high_,low_,close_,volume_,marketcap_,date_pred,close__pred
3897,xrp,2019-06-07,614,0.422013,0.431088,0.417467,0.425189,1.920558e+09,1.788768e+10,2019-07-07,0.396537
345,ada,2018-09-12,346,0.070278,0.070416,0.061445,0.066772,6.039406e+07,1.673668e+09,2018-10-12,0.073637
2335,eth,2019-08-30,698,169.181709,170.585671,166.916342,168.554425,5.840483e+09,1.815741e+10,2019-09-29,169.837804
1109,ada,2020-10-15,1110,0.107265,0.107668,0.104881,0.106546,3.894583e+08,3.348286e+09,2020-11-14,0.104442
1075,ada,2020-09-11,1076,0.096686,0.097021,0.093000,0.096399,8.815262e+08,2.997408e+09,2020-10-11,0.106551
1340,ada,2021-06-03,1341,1.760000,1.880000,1.720000,1.840000,3.157680e+09,5.690709e+10,2021-07-03,1.405568
1289,ada,2021-04-13,1290,1.330000,1.441545,1.293779,1.430000,4.370349e+09,4.291033e+10,2021-05-13,1.941790
1171,ada,2020-12-16,1172,0.154004,0.167839,0.149404,0.167839,6.445788e+08,4.938921e+09,2021-01-15,0.301958
1571,ada,2022-01-20,1572,1.338726,1.423573,1.254234,1.258814,3.476250e+09,4.263993e+10,2022-02-19,0.998086
325,ada,2018-08-23,326,0.089662,0.093429,0.088985,0.092304,4.402221e+07,2.353493e+09,2018-09-22,0.083236


In [47]:
predict_df_30_day.sample(15)

,type_crypto,date_orig,close_orig,date_pred,close_pred
1821,eth,2018-03-04,868.121552,2018-04-03,850.273395
900,ada,2020-03-20,0.029362,2020-04-19,0.101962
3800,xrp,2019-01-01,0.363641,2019-01-31,0.430779
4259,xrp,2020-04-04,0.181022,2020-05-04,0.342119
3994,xrp,2019-07-14,0.306122,2019-08-13,0.406905
4573,xrp,2021-02-12,0.609601,2021-03-14,0.642652
4442,xrp,2020-10-04,0.247686,2020-11-03,0.382377
634,ada,2019-06-28,0.089715,2019-07-28,0.143256
3227,eth,2022-01-08,3099.625501,2022-02-07,3070.211471
678,ada,2019-08-11,0.055176,2019-09-10,0.118820


## Send to Postgres

In [49]:
predict_df_60_day.to_sql(name='predict_df_60_day', con=engine)

In [50]:
predict_df_30_day.to_sql(name='predict_df_30_day', con=engine)

In [51]:
clean_df_60_day.to_sql(name='clean_df_60_day', con=engine)

In [52]:
clean_df_30_day.to_sql(name='clean_df_30_day', con=engine)